# How to use Google calendar API?

How to authorize Google calendar?



In [ ]:
var {google} = require('googleapis');
var importer = require('../Core');
var authorize = importer.import('google oauth token client');

// Authorize a client with the loaded credentials, then call the
// Google Calendar API.
var oauth2Client;
function authorizeCalendar(options = {}) {
    if(options.auth) {
        return Promise.resolve(google.calendar({version: 'v3', auth: options.auth}))
    }
    return authorize(options.scopes || [
            'https://www.googleapis.com/auth/calendar'])
        .then(c => {
            options.auth = c;
            return google.calendar({version: 'v3', auth: c});
        })
}

module.exports = authorizeCalendar;



How to list events?

In [ ]:
var importer = require('../Core');
var {
    authorizeCalendar,
    correctTimeLimits,
    correctCalendarId
} = importer.import([
    'authorize google calendar',
    'correct dates time',
    'lookup calendar name'
]);

function processResult(response, options) {
    var resultEvents = []; // reset the array
    var events = response.data.items;
    if (events.length === 0) {
        const q = Object.assign({}, options);
        delete q.auth;
        console.log('No events found for ' + JSON.stringify(q));
    } else {
        for (var i = 0; i < events.length; i++) {
            var event = events[i];
            var start = event.start.dateTime || event.start.date;
            resultEvents[resultEvents.length] = {
                start: new Date(start), event: event
            };
        }
    }
    return resultEvents;
};


/**
 * Lists the next 10 events on the user's primary calendar.
 *
 * @param {google.auth.OAuth2} auth An authorized OAuth2 client.
 */
function listEvents(options = {calendarId: 'primary'}) {
    let calendar, events = [];
    return authorizeCalendar(options)
        .then(c => calendar = c)
        .then(() => correctTimeLimits(options))
        .then(() => correctCalendarId(options))
        .then(() => {
            var data = Object.assign({}, options, {
                maxResults: 1000,
                singleEvents: true,
                orderBy: 'startTime'
            });
            return new Promise((resolve, reject) => {
                calendar.events.list(data, (err, response) => {
                    if (err) reject(err);
                    try {
                        return resolve(response);
                    } catch (e) {
                        return reject(e);
                    }
                });
            })
        })
        .then(response => {
            events = events.concat(processResult(response, options));
            //console.log(response)
            if(response.data.nextPageToken) {
                console.log('paging ' + response.data.nextPageToken)
                return listEvents(Object.assign({}, options, {pageToken: response.data.nextPageToken}))
                    .then(e => events.concat(e));
            }
            return events;
        })
};
module.exports = listEvents;



Run todays calendar events?



In [ ]:
var importer = require('../Core');
var {JSDOM} = require('jsdom');
var {
    listEvents,
    storeResult
} = importer.import([
    'list events',
    'import google calendar api',
    'store rpc result'
]);

var options = {
    calendarId: 'commands'
};

// test Google calendar API?
function runTodaysEvents(calendar) {
    if(calendar) {
        options.calendarId = calendar;
    }
    return listEvents({
            auth: options.auth,
            calendarId: options.calendarId,
            timeMin: '12 AM today',
            timeMax: 'next hour today'
        })
        // filter processed
        .then(events => {
            // TODO: create object with property "already" and filter out like other RPC methods?
            // determine if the event has already been run today by matching it with the result
            const commandEvent = events
                .filter(e => {
                    const matches = events
                        .filter(m => 'Result: ' + e.event.summary === m.event.summary
                            && e.event.start.dateTime === m.event.start.dateTime);
                    return e.event.summary.indexOf('Result:') === -1 && matches.length === 0
                })[0]
            if (typeof commandEvent === 'undefined') throw new Error('No events!');
            // parse parameters from event description
            var parameters;
            try {
                var dom = new JSDOM('<body>' + (commandEvent.event.description || '""').replace(/<br\/?>/igm, '\n') + '</body>');
                parameters = JSON.parse(dom.window.document.body.textContent);
            } catch ( e ) {
                parameters = (commandEvent.event.description || '').trim().split(/\n/ig);
            }
            return {
                already: false, // filtered out above
                body: parameters,
                name: 'Today\'s events',
                command: commandEvent.event.summary.trim(),
                date: new Date(commandEvent.event.start.dateTime),
                id: commandEvent.event.summary.trim(),
                circles: ['Function', 'Selenium'],
                result: importer.interpret(commandEvent.event.summary.trim()),
                allowed: true // TODO: use RPC filter because it's safer?
            };
        })
        // use rpc
        .then(f => storeResult(f, options.calendarId))
        .catch(e => console.log(e))
}

module.exports = runTodaysEvents;

// node -e "require('/Users/briancullinan/jupytangular2/Core').import('heartbeat')('run todays calendar events')"

// echo "require('/Users/briancullinan/jupytangular2/Core').import('scrape facebook events').then(runner => runner()).then(e=>{console.log(e); process.exit(e) }).catch(e=>{console.log(e); process.exit(e) });" | node



test calendar events


In [ ]:
var importer = require('../Core');
var {
    listEvents,
    correctCalendarId
} = importer.import([
    'list events',
    'import google calendar api',
    'lookup calendar name'
]);

var options = {};
describe('calendar service', () => {
    beforeEach(() => {
    })
    
    it('should list events from around today', (done) => {
        return correctCalendarId(Object.assign(options, {
            calendarId: 'commands'
        }))
        .then(() => listEvents({
            auth: options.auth,
            calendarId: options.calendarId,
            timeMin: '12 AM today',
            timeMax: 'next hour today'
        }))
        .then(r => {
            assert(r.length > 0)
            done()
        })
    })
})


Correct calendar dates for timeMax and timeMin?


In [ ]:
var importer = require('../Core');
var chrono = require('chrono-node');
var ISODateString = importer.import('convert date to ISO');

function correctTimeLimits(options) {
    if (typeof options.timeMin !== 'undefined') {
        options.timeMin = ISODateString(
            chrono.parseDate(options.timeMin));
    }
    if (typeof options.timeMax !== 'undefined') {
        options.timeMax = ISODateString(
            chrono.parseDate(options.timeMax));
    }
    return options;
};
module.exports = correctTimeLimits;



Lookup calendar id by name or id?



In [ ]:
var importer = require('../Core');
var authorizeCalendar = importer.import('authorize google calendar');
var assert = require('assert');
var util = require('util');

var calendarList = [], lastCalendar;

function filterCalendar(options) {
    var rexexp = new RegExp(options.calendarId, 'ig');
    var matches = calendarList
        .filter(c => c.id == options.calendarId);
    if (matches.length == 0) {
        matches = calendarList
            .filter(c => c.summary.match(rexexp));
    }
    assert(matches.length > 0, 'something is seriously wrong with the calendarId ' + JSON.stringify(options, null, 4) + JSON.stringify(calendarList, null, 4));
    if (lastCalendar !== matches[0].summary) {
        lastCalendar = matches[0].summary;
        console.log('Using calendar: ' + matches[0].summary
            + ' - ' + matches[0].id);
    }
    options.calendarId = matches[0].id;
    return Promise.resolve(options);
}

function correctCalendarId(options) {
    if (typeof options.calendarId === 'undefined' || options.calendarId === 'primary') {
        return Promise.resolve(Object.assign(options, {
            calendarId: 'primary'
        }))
    }
    if (calendarList.length > 0) {
        return filterCalendar(options);
    }
    return (calendarList.length == 0
        ? authorizeCalendar()
            .then(calendar => util.promisify(calendar.calendarList.list.bind(calendar))())
        : Promise.resolve(calendarList))
        .then(r => {
            calendarList = (r.data || {}).items || [];
            return filterCalendar(options);
        })
        .catch(e => console.log(e))
};

module.exports = correctCalendarId;



days events?

get days events?


In [ ]:
var chrono = require('chrono-node');
var assert = require('assert');
var importer = require('../Core');
var {
    ISODateString,
    listEvents 
} = importer.import([
    'convert date iso',
    'list events'
]);

function getDaysEvents(startDate, options) {
    var eventCache = {}; // TODO: fix this, move it outside, only update if calendar is modified?
    startDate = chrono.parseDate(startDate + '');
    var day = new Date(startDate);
    day.setHours(0, 0, 0);
    var dayEnd = new Date(day);
    dayEnd.setDate(dayEnd.getDate() + 1);
    var todaysEvents;
    assert(options.calendarId, 'calendarId must be set by now! ' + JSON.stringify(options, null, 4));
    if(typeof eventCache[options.calendarId] === 'undefined') {
        eventCache[options.calendarId] = {};
    }
    if(typeof eventCache[options.calendarId][day.getTime()] !== 'undefined') {
        todaysEvents = Promise.resolve(eventCache[options.calendarId][day.getTime()])
    } else {
        console.log({
            timeMin: ISODateString(new Date(day.getTime())),
            timeMax: ISODateString(new Date(dayEnd.getTime()))
        });
        todaysEvents = listEvents({
            auth: options.auth,
            calendarId: options.calendarId,
            timeMin: ISODateString(new Date(day.getTime())),
            timeMax: ISODateString(new Date(dayEnd.getTime()))
        })
    }
    return todaysEvents
        .then(m => {
            assert(eventCache[options.calendarId], 'something is seriously wrong with the calendar service "' + options.calendarId + '" ' + JSON.stringify(eventCache, null, 4));
            eventCache[options.calendarId][day.getTime()] = m;
            return m.filter(e => e.event.deleted !== true)
        })
}
module.exports = getDaysEvents;


Update create merge delete event?



In [ ]:
var {google} = require('googleapis');
var util = require('util');
var importer = require('../Core');
var chrono = require('chrono-node');
var {JSDOM} = require('jsdom');
var getDaysEvents = importer.import('days events');

var {
    authorizeCalendar, ISODateString, correctCalendarId
} = importer.import([
    'authorize google calendar',
    'convert date iso',
    'lookup calendar name',
])

function updateEvent(event, options) {
    var calendar;
    
    return authorizeCalendar()
        .then(c => calendar = c)
        .then(() => correctCalendarId(options))
        .then(() => getDaysEvents(event.start.dateTime, options))
        .then(m => {
            const actionsArray = [];
            const matches = m.filter(match => !match.event.deleted
                && match.event.summary.toLowerCase().trim() === event.summary.toLowerCase().trim()
                && Math.abs(Math.round(new Date(match.event.start.dateTime).getTime() / 1000 / 60)
                    - Math.round(new Date(event.start.dateTime).getTime() / 1000 / 60)) < 32);
            console.log('Matching ' + matches.length);
        
            // TODO: check for existing event if the ID is already set?
            if (matches.length > 0) {
                var unique = [];
                try {
                    let descriptions = JSON.parse(event.description) || [];
                    descriptions = descriptions.concat(
                        ...matches.map(match => {
                            try {
                                return JSON.parse(match.event.description)
                            } catch (e) {
                                return [];
                            }
                        }));
                    const urls = descriptions.map(d => (d || {}).url);
                    unique = descriptions.filter((d, i) => urls.indexOf((d || {}).url) === i);
                } catch (e) {
                    unique = [event.description];
                }
                // TODO: make sure there are no duplicates
                // TODO: move this in to some parsing utility?
                // TODO: deep compare instead of just comparing "url" property?
                // support for objects if there's only one
                unique = unique.length === 1 ? unique[0] : unique;
                // patch the first match
                var newDescription = typeof unique === 'string' ? unique : JSON.stringify(unique, null, 4);
                
                if(matches[0].event.description !== newDescription) {
                    actionsArray.push(util.promisify(
                        calendar.events.patch.bind(calendar.events, {
                            eventId: matches[0].event.id,
                            calendarId: options.calendarId,
                            auth: options.auth,
                            resource: {
                                description: newDescription,
                                colorId: event.colorId
                            }
                        }))());
                }

                // TODO: delete the rest
                for (const match of matches.slice(1)) {
                    match.event.deleted = true;
                    actionsArray.push(util.promisify(
                        calendar.events.delete.bind(calendar.events, {
                            eventId: match.event.id,
                            calendarId: options.calendarId,
                            auth: options.auth
                        }))());
                }
            } else {
                console.log('adding event ' + event.summary)
                actionsArray.push(util.promisify(
                    calendar.events.insert.bind(calendar.events, {
                        calendarId: options.calendarId,
                        auth: options.auth,
                        resource: event
                    }))());
            }
            return importer.runAllPromises(actionsArray);
        })
        .catch(e => console.log(e))
};
module.exports = updateEvent;


create new calendar event?


In [ ]:
var importer = require('../Core');
var {google} = require('googleapis');
var util = require('util');
var {
    correctCalendarId
} = importer.import([
    'lookup calendar name',
    'import google calendar api'
]);

function createNewEvent(summary, description, options) {
    const now = new Date();
    options = options || {};
    const event = {
        summary: summary,
        description: description,
        start: {dateTime: new Date(now.getTime())},
        end: {dateTime: new Date(now.getTime() + 30 * 60 * 1000)}
    }
    if(typeof summary === 'object') {
        Object.assign(event, summary);
        options = description;
    }
    return correctCalendarId(options)
        .then(() => util.promisify(
            calendar.events.insert.bind(calendar.events, {
                calendarId: options.calendarId,
                auth: options.auth,
                resource: event
            })))
}
module.exports = createNewEvent;

